<a href="https://colab.research.google.com/github/adityasingh1993/NLP-ANN/blob/master/Toxic_Comment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install dependencies






In [1]:
!pip install kaggle
!pip install contractions
!pip install inflect

**Upload the kaggle.json file**

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adityasinghce92","key":"e79766a1206ef2fd74c1f97860d95521"}'}

**Manage permission for kaggle**

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

**Download Dataset**

In [6]:
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

 46% 12.0M/26.3M [00:00<00:00, 122MB/s]
100% 26.3M/26.3M [00:00<00:00, 129MB/s]
  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 93.6MB/s]
 51% 12.0M/23.4M [00:00<00:00, 124MB/s]
100% 23.4M/23.4M [00:00<00:00, 150MB/s]
  0% 0.00/1.46M [00:00<?, ?B/s]
100% 1.46M/1.46M [00:00<00:00, 99.4MB/s]


**Download Glove word embeddings**

In [0]:
	
!wget --header="Host: downloads.cs.stanford.edu" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-IN,en;q=0.9,en-GB;q=0.8,en-US;q=0.7,hi;q=0.6" --header="Cookie: _ga=GA1.2.778886217.1559722518; _mkto_trk=id:194-OCQ-487&token:_mch-stanford.edu-1560092631271-53052; _gid=GA1.2.1948571120.1579118412" --header="Connection: keep-alive" "http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip" -O "glove.6B.zip" -c


--2020-01-27 15:40:30--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  2.00MB/s    in 6m 28s  

2020-01-27 15:46:59 (2.12 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]





**Unzipe the downloaded embeddings file**


In [0]:
!unzip glove.6B.zip 

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


**Import the required dependency**

In [0]:
import pandas as pd
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

from nltk.stem import LancasterStemmer, WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional,CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPool1D

**Kaggle Starter Code**

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

**Check The available files**

In [76]:
!ls

kaggle.json		   submission.csv   test_labels.csv.zip
sample_data		   test.csv	    train.csv
sample_submission.csv	   test.csv.zip     train.csv.zip
sample_submission.csv.zip  test_labels.csv


**Unzip the dataset**

In [77]:
!unzip train.csv.zip
!unzip test.csv.zip
!unzip test_labels.csv.zip
!unzip sample_submission.csv.zip

Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

KeyboardInterrupt: ignored

**Load The Datset**

In [0]:
dataframe_train=pd.read_csv("train.csv")
dataframe_test=pd.read_csv("test.csv")
dataframe_test_label=pd.read_csv("test_labels.csv")

**Basic preprocessing function**

In [0]:
import re
def replace_contractions(text):
    return contractions.fix(text)
def cleanText(text):
    text = text.strip().replace("\n", " ").replace("\r", " ")
    text = replace_contractions(text)
    #text = replace_links(text, "link")
    #text = remove_numbers(text)
    text = re.sub(r'[,!@#$%^&*)(|/><";:.?\'\\}{]',"",text)
    text = text.lower()
    text=text.strip()
    return text

**Take the texts from test and train dataset, clean it , club it so that we can make vocabulory**

In [0]:
listsentence=list(dataframe_train["comment_text"])
listsentence_test=list(dataframe_test["comment_text"])

In [0]:
listcleansentence=[]
for i in listsentence:
    listcleansentence.append(cleanText(i))

In [0]:
listcleansentence_test=[]
for i in listsentence_test:
    
    try:
        listcleansentence_test.append(cleanText(i))
    except:
        listcleansentence_test.append(i)

**Combining together to make vocabulory**

In [0]:
allsentences=listcleansentence_test+listcleansentence
words=" ".join(allsentences).split(" ")
word2idx={w:i for i,w in enumerate(words)}

**Tokenize the sentence so that we can get our input feature for our model**

In [0]:
tokenizer = Tokenizer(num_words=len(word2idx))
tokenizer.fit_on_texts(allsentences)
input_sequence = tokenizer.texts_to_sequences(listsentence)


**Pad the data so that our inputs will have same length**

In [0]:
inputx = pad_sequences(input_sequence, maxlen=3500, dtype='int32', padding='post', truncating='post')

In [0]:
## Was tryng to use glove for word embedding,  but accuracy was not good and lots of words were missing around 13k

# def glove_100d_dictionary():
#   embeddings_index = {}
#   f = open('glove.6B.100d.txt')
#   for line in f:
#       values = line.split()
#       word = values[0]
#       coefs = np.asarray(values[1:], dtype='float32')
#       embeddings_index[word] = coefs
#   f.close()
#   return embeddings_index
# listofabsentword=[]
# def embedding_matrix_creater(embedding_dimention):
#   embeddings_index=glove_100d_dictionary()
#   embedding_matrix = np.zeros((len(word2idx) + 1, embedding_dimention))
#   for word, i in word2idx.items():
#       embedding_vector = embeddings_index.get(word)
#       if embedding_vector is not None:
#           # words not found in embedding index will be all-zeros.
#           try:
#             embedding_matrix[i] = embedding_vector
#           except:
#             listofabsentword.append(word)
#             print("exception===>",word)
#             pass
#   return embedding_matrix

#   def embedding_layer_creater(VOCAB_SIZE, EMBEDDING_DIM, MAX_LEN, embedding_matrix):
  
#   embedding_layer = Embedding(input_dim = VOCAB_SIZE, 
#                               output_dim = EMBEDDING_DIM,
#                               input_length = MAX_LEN,
#                               weights = [embedding_matrix],
#                               trainable = False)
#   return embedding_layer

# embedding_matrix=embedding_matrix_creater(100)
# embedding_layer = embedding_layer_creater(maxvocab, 100, 3500, embedding_matrix)

**y is our target class**

In [0]:
y=dataframe_train.iloc[:,2:].values

In [0]:
listy=list(dataframe_train.iloc[:,2:])

**Architecture of our model**

In [0]:
inputs = Input(shape=(3500,))
emb_layer = Embedding(input_dim = 506004, output_dim = 100,input_length = 3500)(inputs)
#emb_layer = Embedding(1000, 100, weights=embedding_matrix)(inp)
#emb_layer=Embedding(input_dim=506003,output_dim=50,input_length=1000)(inp)

In [0]:
lstm_layer = Bidirectional(CuDNNLSTM(50, return_sequences=True))(emb_layer)
maxpool_layer = GlobalMaxPool1D()(lstm_layer)
dense_layer = Dense(50, activation="relu")(maxpool_layer)
dropout_layer = Dropout(0.5)(dense_layer)
dense_layer2 = Dense(6, activation="sigmoid")(dropout_layer)
model = Model(inputs=inputs, outputs=dense_layer2)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [82]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3500)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 3500, 100)         50600400  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 3500, 100)         60800     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 306 

**Was splitting the data for train and test**

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(inputx,y)

**Since our class is very imbalance, i was trying to use class weight, (experimental), din't use in my model**

In [0]:
classlist=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate']
from sklearn.utils import class_weight
classweigh=class_weight.compute_class_weight("balanced",np.array(classlist),listy)

In [84]:
classweigh

array([1., 1., 1., 1., 1., 1.])

In [85]:
np.unique(y_train)

array([0, 1])

In [86]:
classweigh

array([1., 1., 1., 1., 1., 1.])

**Training the model, basic configuration**

In [0]:
model.fit(inputx,y,batch_size=32,verbose=1,epochs=10,validation_split=0.1,workers=3, use_multiprocessing=True)

Train on 143613 samples, validate on 15958 samples
Epoch 1/10
143613/143613 [==============================] - 1438s 10ms/step - loss: 0.0716 - acc: 0.9764 - val_loss: 0.0504 - val_acc: 0.9810
Epoch 2/10
143613/143613 [==============================] - 1476s 10ms/step - loss: 0.0449 - acc: 0.9831 - val_loss: 0.0478 - val_acc: 0.9821
Epoch 3/10
143613/143613 [==============================] - 1502s 10ms/step - loss: 0.0382 - acc: 0.9850 - val_loss: 0.0511 - val_acc: 0.9814
Epoch 4/10
143613/143613 [==============================] - 1503s 10ms/step - loss: 0.0333 - acc: 0.9867 - val_loss: 0.0536 - val_acc: 0.9804
Epoch 5/10
 85568/143613 [================>.............] - ETA: 9:37 - loss: 0.0295 - acc: 0.9879

**Evaluate our Model on ROC score**

In [0]:
from sklearn.metrics import roc_curve,roc_auc_score
ypredtest=model.predict(X_test)
roc_auc_score ( y_test , ypredtest,average="macro")


**do the prediction for the test dataset, download the submission.csv and upload it n kaglle, this model was giving me score of 95.8%**

In [0]:
classlist=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate']
input_sequence_test=tokenizer.texts_to_sequences(listsentence_test)
inputx_test=pad_sequences(input_sequence_test, maxlen=3500, dtype='int32', padding='post', truncating='post')
listres=[]
ypred=model.predict(inputx_test)
sampledf=pd.read_csv("sample_submission.csv")
sampledf[classlist]=ypred
sampledf.to_csv("submission.csv", index=False)
from google.colab import files
files.download("submission.csv") 

In [50]:
!ls

kaggle.json		   submission.csv   test_labels.csv.zip
sample_data		   test.csv	    train.csv
sample_submission.csv	   test.csv.zip     train.csv.zip
sample_submission.csv.zip  test_labels.csv
